# Pytorch Basic  
![](_2023-04-17-13-59-20.png)  

작성자: 김국진 kukjinkim@korea.ac.kr  



In [ ]:
!pip3 install torch --index-url https://download.pytorch.org/whl/cu117

![](_2023-04-17-13-37-23.png)  

과연 ChatGPT는 Pytorch로 구현되었을까요?  


# Contents
- Class Review 
- Data representation with tensor <-
- Tensor Manipulation <-
- Dataset, DataLoader
- Regresseion 
- Clasification

2016년 이후로 딥러닝과 강화학습은 정말 빠른 속도로 발전하여 현재의 ChatGPT와 같은 혁신적인 결과물을 탄생시켰습니다.  
본 실습에서는 아주 널리 쓰이고 있는 Pytorch 딥러닝 프레임워크에 대해 본격적으로 실습해보겠습니다.  

### 실습을 위해 GPU가 필요합니다. 노트북에 GPU가 없다면, 모든 device 코드를 cpu로 바꿔서 실행하면 해결할 수 있습니다. 또는 Colab 상에서 실습을 진행해주세요. 

![img](2023-04-25-21-42-12.png)

![](2023-04-25-21-43-14.png)

# 2. Data representation with tensor  
선형대수에서는 스칼라, 벡터, 행렬, 텐서의 개념에 대해 배웁니다.  
선형대수 책과 많은 머신러닝 도서들에서는 아래와 같은 기호로 이들을 표기합니다.  
여러 모듈과 메소드를 통해 파이토치에서 어떻게 텐서를 조작하는지 알아보겠습니다  

$$\text{Scalar} : x\quad 0 \  \text{(0-dimension) }$$
$$\text{Vector} : \mathbf x \quad n \ \text{ (1-dimension)}$$
$$\text{Matrix} : \mathbf X \quad m \times n \  \text {(2-dimension)}$$
$$\text{Tensor} : \mathcal {X} \quad l \times m \times n \times \dots \text {(more 3-dimension)} $$

![](2023-04-25-21-44-38.png)

(https://furkangulsen.medium.com/what-is-a-tensor-ce8e78835d08)

프로그래밍 과목의 초입에서 배우는 데이터의 타입을 떠올려보겠습니다.  
대표적으로 다음과 같습니다. 
1. Int  
2. Double  
3. Float  

마찬가지로 torch의 여러가지 텐서모듈을 통해 위의 데이터타입을 가지는 텐서를 생성할 수 있습니다.

In [9]:
import torch
from torch import tensor
from torch import FloatTensor as ftensor
from torch import DoubleTensor as dtensor
from torch import IntTensor as itensor

#### 2.1 Scalar
`torch.tensor(data)`는 data array을 입력으로 받아서 pytorch의 tensor object를 리턴합니다.  
  
  
텐서 객체는 다음과 같은 형태로 할당하게 됩니다.    
`tensor = torch.tensor(data, dtype=float32, device='cpu')`   
  

텐서 객체를 생성하기 위한 파라미터와 인자는 data, dtype, device가 있습니다.   
data에는 리스트나 numpy 배열이 들어갈 수 있으며,  
device에는 cpu나 cuda:0와 같이 gpu 인덱스가 들어갈 수 있습니다. 

기본적으로는 argument를 주지 않으면 tensor는 cpu로 계산되고, data type은 float32나 int64로 설정됩니다. 

In [23]:
tensor1 = torch.tensor(1.0)
print(tensor1)
print(tensor1.device)
print(tensor1.dtype)

tensor2 = torch.tensor(2, device='cuda')
print(tensor2)
print(tensor2.device)
print(tensor2.dtype)

# For plotting value
print(tensor1.item())
print(tensor2.item())




tensor(1.)
cpu
torch.float32
tensor(2, device='cuda:0')
cuda:0
torch.int64
1.0
2


여기서 기억해야할 것들은 다음과 같습니다. 
- 텐서 오브젝트의 dtype과 device는 모두 통일시켜야 한다
- 즉, 입력 텐서의 데이터타입과 모델 가중치 텐서의 데이터 타입이 같아야 한다. 
- 또한, 입력 텐서와 모델 가중치 텐서의 계산장치가 같아야 한다.

위 사항들을 지켜지지 않았을 때는 pytorch가 내부적으로 device와 dtype을 변환해서 연산을 수행해줍니다. 

In [26]:
tensor3 = tensor1 + tensor2
print(tensor3)
print(tensor3.device)
print(tensor3.dtype)

tensor(3., device='cuda:0')
cuda:0
torch.float32


#### 2.2 Vector

In [27]:
import numpy as np
vec = np.array([1, 2, 3, 4])

numpy에서는 시퀀스 데이터(리스트, 튜플)을 넣어서 벡터나 텐서를 표현합니다.  
`torch.tensor(data)`에서 data에 들어갈 수 있는 컨테이너는 List, Tuple, Numpy array입니다.  
파이토치에서 텐서 객체를 만드는 방법은 기존의 numpy와 매우 유사합니다.  



In [29]:
vec1 = ftensor([1.0, -1.1, 3.9]) # list input
vec2 = dtensor((1.0, -1.1, 3.9)) # Tuple input
vec3 = tensor(np.array([1, 2, 3, 4])) # numpy ary input
vec4 = tensor([1.0, -1.1, 3.9], dtype=int)

print(vec1)
print(vec1.dtype)
print(vec2)
print(vec3)
print(vec4)

tensor([ 1.0000, -1.1000,  3.9000])
torch.float32
tensor([ 1.0000, -1.1000,  3.9000], dtype=torch.float64)
tensor([1, 2, 3, 4], dtype=torch.int32)
tensor([ 1, -1,  3])


#### 2.3 Matrix  
이제 행렬 텐서를 만들어보겠습니다.  
데이터타입을 행렬로 다루기 시작한다면 병렬처리를 위해 device를 gpu로 지정하는게 좋습니다.  
이제 데이터를 일일히 생성하기 힘드므로 rand를 사용하겠습니다.  
rand의 결과가 같도록 만들기 위해 시드를 고정하겠습니다.  

In [30]:
torch.manual_seed(1) # torch.manual_seed(seed) fixing random seed

In [33]:
mat = torch.rand([2, 2], device='cuda')
print(mat)
print(mat.device)


tensor([[0.8903, 0.0275],
        [0.9031, 0.5386]], device='cuda:0')
cuda:0


#### 2.4 Tensor
이제 3차원 이상의 텐서를 만들어보겠습니다.  
`to(dtype=dtype, device=device)` : 텐서 내의 메소드로 원하는 데이터 타입, 원하는 디바이스의 텐서를 반환해줍니다. 


In [41]:
torch.manual_seed(2)
# T = torch.rand([64, 64, 3], device='cuda:0')
T = torch.rand([64, 64, 3], device='cpu')
T = T * 256
print(T[0:1])
print(T.dtype)

T = T.to(int) 
T = T.to(device='cuda:0')
print(T[0:1])
print(T.dtype)

tensor([[[157.3619,  97.5393, 163.1013],
         [121.4620, 182.6800, 158.4733],
         [113.2884,  24.5168, 157.2242],
         [ 14.6759, 144.8228, 136.5066],
         [ 99.8530, 232.6652, 136.5423],
         [181.0788, 182.1689,  52.4888],
         [ 78.7924, 251.0999,   2.6269],
         [119.3065, 117.8560, 218.7916],
         [115.8311, 161.7047, 121.8555],
         [ 56.3268,  55.4509,  65.8133],
         [ 11.7242,  44.9303, 158.1247],
         [212.2427, 134.3056,  69.3261],
         [184.2539,  78.8643,  99.6456],
         [ 57.8295,  87.8077,   9.3952],
         [182.6163, 177.7710, 153.4251],
         [190.8456, 182.2505, 133.6631],
         [141.5559, 137.7742, 196.3005],
         [213.9836, 219.9226, 202.1895],
         [ 96.7924, 122.3017, 101.9875],
         [202.4617, 142.2154, 246.4687],
         [192.9206,  18.6036, 165.4441],
         [250.9919, 241.6952, 125.9885],
         [170.4645,   7.9294,  87.1935],
         [190.4178,  11.3967, 239.5140],
         [ 43.82

# Pop Quiz 1 (Easy)  
1) !nvidia-smi 를 통해 현재 gpu의 메모리 용량과 현재 사용량을 확인하세요
2) 크기가 (1000, 64, 64, 3)인 4차원 랜덤 텐서를 생성해서 gpu 메모리에 할당하세요
3) 다시 한 번 더 `!nvidia-smi` 를 통해 gpu 메모리 사용량이 얼마나 늘었는지 계산해보세요

#### 만약 노트북에 GPU가 없다면 CPU를 device로 설정하고 작업관리자를 통해 메모리 사용량을 확인해보세요. 

In [ ]:
!nvidia-smi

In [ ]:
4d_tensor = pass

In [4]:
!nvidia-smi


# 3. Tensor manipulation

선형대수에서는 두 벡터의 덧셈, 뺄셈, 내적, 외적 등의 연산 방법을 배우고더 행렬의 덧셈, 뺄셈, 곱셈, 역행렬의 계산 등의 행렬 연산을 배웁니다. 이러한 모든 벡터, 행렬, 텐서 연산방법을 마찬가지로 pytorch에서도 수행할 수 있습니다.  

#### 3.1 Vector, Matrix, Tensor Operation

In [54]:
# Vector Inner Product
vec1 = tensor([1, 2, 3])
vec2 = tensor([4, 5, 6])
# 벡터의 dot product를 위해서는 torch.dot 메소드 또는 tensor.dot 메소드를 사용합니다.
vec3 = vec1.dot(vec2) # 1 * 4 + 2 * 5 + 3 * 6
print(vec3)




tensor(32)


In [56]:
# Vector Cross Product
crosse_vec = torch.cross(vec1, vec2) # 벡터의 길이가 3차원이어야 합니다.
print(crosse_vec)

tensor([-3,  6, -3])


In [57]:
# Vector Outer Product
outer_vec = torch.outer(vec1, vec2) # 벡터의 외적을 계산합니다. 출력은 행렬입니다. 
print(outer_vec)


tensor([[ 4,  5,  6],
        [ 8, 10, 12],
        [12, 15, 18]])


In [59]:
# * : 에스터리스크(asterisk) 이 기호는 element wise product를 수행합니다.
vec4 = vec1 * vec2
print(vec4) # [4, 10, 18]



tensor([ 4, 10, 18])


In [60]:
# 모든 텐서는 에스터리스크 기호를 통해 원소곱을 수행할 수 있습니다.  
mat1 = tensor([[1, 2], 
              [3, 4]])
mat2 = tensor([[5, 6], 
              [7, 8]])
print(mat1 * mat2) # 5, 12, 21, 32

tensor([[ 5, 12],
        [21, 32]])


In [62]:
# at sign @ : 골뱅이 기호는 행렬곱을 수행하게 됩니다. 
# matmul 함수를 통해서도 행렬곱을 계산할 수 있다. 

mat4 = mat1 @ mat2
mat5 = mat1.matmul(mat2)
print(mat4)
print(mat5)

ten1 = tensor([[[1, 2], 
              [3, 4]],
                [[5, 6], 
              [7, 8]]])
ten2 = tensor([[[1, 2], 
              [3, 4]],
                [[5, 6], 
              [7, 8]]])

a = ten1.matmul(ten2)
b = ten1 @ ten2
print(a)
print(b)


tensor([[19, 22],
        [43, 50]])
tensor([[19, 22],
        [43, 50]])
tensor([[[  7,  10],
         [ 15,  22]],

        [[ 67,  78],
         [ 91, 106]]])
tensor([[[  7,  10],
         [ 15,  22]],

        [[ 67,  78],
         [ 91, 106]]])


In [85]:
# 항등행렬 .eye(shape)를 통해 만들 수 있습니다.
eye_tensor = torch.eye(2)
print(eye_tensor.shape)

# 행렬의 대각성분 torch.diag(matrix)
v = torch.randn([4,4])
print(v)
diag_vector = torch.diag(v)
print(diag_vector)

torch.Size([2, 2])
tensor([[-0.7201, -2.3542, -2.1832, -1.1991],
        [-0.4095, -1.5249,  1.0711,  0.4676],
        [-0.4792, -0.2741, -0.2697,  0.9947],
        [ 2.8023,  0.1025,  0.7995, -0.7051]])
tensor([-0.7201, -1.5249, -0.2697, -0.7051])


In [64]:
# tensor dot도 존재하지만 잘 쓰이지는 않습니다. 
print(torch.tensordot(ten1, ten2))

tensor([[ 50,  60],
        [114, 140]])


#### 3.2 * **Tensor dimension manipulation** * (Very important!)  
딥러닝 모델에 데이터를 입력하기 위해 텐서의 차원을 정말 잘 다루어야 합니다.   
이때 차원을 조작하는 메소드로는 다음과 같습니다.
1) 차원 축소, 확장 :  `squeeze(), unsqueeze()` 차원의 크기가 1인 차원을 제거하거나 추가합니다. 
2) 차원 교환 : `transpose(), permute()` : 차원의 순서를 바꿉니다. 보통 3차원 이상의 텐서를 다룰 때 사용합니다.      
3) 텐서 모양 변경 : `flatten(), view(), reshape() ` 
4) 텐서 합성 및 적재 : `cat(), stack()`  

#### 3.2.1 Dimension reduction, extension
`tensor.squeeze(axis)`또는 `torch.squeeze(axis)` 함수는 axis 값에 해당하는 특정 축을 제거합니다.
`tensor.unsqueeze(axis)` 또는 `torch.unsqueeze(axis)`함수는  axis 값에 해당하는 축에 차원을 추가합니다. 

In [88]:
vec1 = tensor([1, 2, 3])
print(vec1.shape)
unsq_vec1 = vec1.unsqueeze(1)
print(unsq_vec1.shape)
unsq_vec1 = unsq_vec1.unsqueeze(1)
print(unsq_vec1.shape)
sq_vec1 = unsq_vec1.squeeze(2)
print(sq_vec1.shape)
sq_vec1 = sq_vec1.squeeze(1)
print(sq_vec1.shape)
sq_vec1 = sq_vec1.squeeze()
print(sq_vec1.shape)





torch.Size([3])
torch.Size([3, 1])
torch.Size([3, 1, 1])
torch.Size([3, 1])
torch.Size([3])
torch.Size([3])


In [89]:
# torch.ones(shape)를 통해 모든 값이 1인 텐서를 쉽게 만들 수 있습니다.
tensor_7d = torch.ones([10, 32, 1, 1, 64, 64, 3])
tensor_7d = torch.zeros([10, 32, 1, 1, 64, 64, 3])


# 차원의 모양을 일일히 입력하기 귀찮을 때 like가 붙은 메소드를 사용합니다.
tensor_7d2 = torch.ones_like(tensor_7d)
tensor_7d3 = torch.zeros_like(tensor_7d)
print(tensor_7d.shape)
print(tensor_7d2.shape)

tensor_6d = tensor_7d.squeeze(axis=2) # shape에서 2번째 인덱스에 해당하는 축을 제거합니다. 
print(tensor_6d.shape) # [10, 32, 1, 64, 64, 3]
tensor_5d = tensor_7d.squeeze()
print(tensor_5d.shape) # [10, 32, 64, 64, 3]



torch.Size([10, 32, 1, 1, 64, 64, 3])
torch.Size([10, 32, 1, 1, 64, 64, 3])
torch.Size([10, 32, 1, 64, 64, 3])
torch.Size([10, 32, 64, 64, 3])


In [87]:
tensor_4d = torch.ones([32, 3, 64, 64])
print(tensor_4d.shape)

tensor_5d = tensor_4d.unsqueeze(0)
tensor_5d2 = tensor_4d.unsqueeze(4) # = -1 
print(tensor_5d.shape)
print(tensor_5d2.shape)


torch.Size([32, 3, 64, 64])
torch.Size([1, 32, 3, 64, 64])
torch.Size([32, 3, 64, 64, 1])


#### 3.2.2 Dimension exchange: ` transpose(), permute()  `
위 메소드들은 차원을 변경할 때 사용합니다. 


In [91]:
matrix = torch.ones([2, 3])
print(matrix.shape)

torch.Size([2, 3])


In [93]:
# torch.transpose(input, dim0, dim1) 는 인풋 텐서의 두 차원을 교환합니다.
# tensor.transpose()를 호출하거나 torch.transpose() 를 통해 사용가능합니다. 
transposed = torch.transpose(matrix, 0, 1)
print(transposed.shape)
transposed2 = matrix.transpose(0, 1)
print(transposed2.shape)

tensor_4d = torch.ones([4, 2, 6, 11])
# 퀴즈 : 0번째, 2번째 차원을 교환하세요
transposed_4d = tensor_4d.transpose(0, 2)
print(transposed_4d.shape) # 6, 2, 4, 11


torch.Size([3, 2])
torch.Size([3, 2])
torch.Size([6, 2, 4, 11])


In [96]:
# torch.permute() : transpose의 상위버전 메소드. 차원 축 순서를 원하는 대로 나열합니다. 

# 텐서 순서의 조정은 직접 해야 한다.

permuted_4d = torch.permute(tensor_4d, [3, 2, 0, 1])
print(permuted_4d.shape)
permuted_4d2 = torch.permute(tensor_4d, (1, 0, 2, 3))
print(permuted_4d2.shape)

torch.Size([11, 6, 4, 2])
torch.Size([2, 4, 6, 11])


# Pop Quiz 2 (Easy)   
tensor_4d의 0번째 축에 잉여차원을 추가하고 `permute()`를 사용해 0번째, 2번쨰 차원을 교환하세요

In [2]:
# Code implementation
pass

#### 3.2.3 Changing the shape of tensor  : `flatten(),  reshape(), view()`  
위 세 가지 메소드는 주로 CNN 레이어와 FC 레이어 사이에서 데이터를 주고 받을 때 자주 사용합니다.   
또는 강화학습에서 입력 텐서의 모양을 변경할 때 자주 사용합니다. 


`torch.flatten()` 메소드는 텐서전체를 1차원 벡터로 변환합니다.


In [100]:
t = tensor([[[1, 2],
            [3, 4]],
           [[5, 6],
           [7, 8]]])
print(t.shape)

torch.Size([2, 2, 2])


In [101]:
flattened_t = torch.flatten(t)
print(flattened_t)
print(flattened_t.shape)

tensor([1, 2, 3, 4, 5, 6, 7, 8])
torch.Size([8])


In [102]:
# 만약에 start_dim, end_dim이 주어지면 해당하는 차원부터 flatten합니다.
flattened_t2 = torch.flatten(t, start_dim=2)
print(flattened_t2)
print(flattened_t2.shape)

tensor([[[1, 2],
         [3, 4]],

        [[5, 6],
         [7, 8]]])
torch.Size([2, 2, 2])


`reshape()` 메소드는 텐서를 원하는 모양을 가지도록 차원을 조정합니다.  
단 이때, 각 차원의 총 형태가 원래의 차원과 일치해야 합니다. ex 8 -> 2, 2, 2 or 4, 2  
-1을 입력하면 해당차원을 자동으로 계산합니다. 



In [108]:
t2 = torch.arange(36)
t2.shape

torch.Size([36])

In [109]:
reshaped_t = t2.reshape(3, 3, 4)
print(reshaped_t.shape)

torch.Size([3, 3, 4])


In [107]:
reshaped_t = t2.reshape(-1, 2) 
print(reshaped_t.shape)

torch.Size([18, 2])


`torch.view(m, ...,)`: reshape와 동일한 기능을 하지만 메모리 관점에서 다르게 동작합니다. 자세한 내용은 documentation을 참고하세요. 


In [116]:
t3 = torch.arange(24940)
viewed_t1 = t3.view(5, 4, -1)
viewed_t2 = t3.reshape(10, -1, 1)
print(viewed_t1.shape)
print(viewed_t2.shape)


torch.Size([5, 4, 1247])
torch.Size([10, 2494, 1])


#### 3.2.3 Stacking and Concatenation : `cat() stack()` 
마지막으로 텐서를 합성하기 위해 cat()과 stack()을 활용합니다.  
`cat()` 두 텐서를 동일한 축을 기준으로 합칩니다. 모델 내부에서 다음 은닉층에 들어가는 텐서들을 조합하기 위해 주로 사용합니다.   
`stack()` 함수는 데이터를 쌓아서 미니배치를 구성할 때 주로 사용합니다. 


In [119]:
# torch.cat() 함수는 두 개 이상의 텐서 시퀀스들을 (특정 축을 기준으로) 합치게 됩니다. 
# 해당 텐서들의 차원은 반드시 같아야 합니다. 
vec1 = tensor([1, 2, 3])
vec2 = tensor([4, 5, 6])
vec3 = tensor([7, 8, 9])
vec4 = torch.cat([vec1, vec2, vec3])
print(vec4)

tensor([1, 2, 3, 4, 5, 6, 7, 8, 9])


In [120]:
# stack() 메소드는 두 개 이상의 텐서들을 특정 축을 기준으로 쌓게 됩니다.
batch_vec = torch.stack([vec1, vec2, vec3], dim=0)
print(batch_vec)
print(batch_vec.shape)

batch_vec = torch.stack([vec1, vec2, vec3], dim=1)
print(batch_vec)
print(batch_vec.shape)

tensor([[1, 2, 3],
        [4, 5, 6],
        [7, 8, 9]])
torch.Size([3, 3])
tensor([[1, 4, 7],
        [2, 5, 8],
        [3, 6, 9]])
torch.Size([3, 3])


# Pop Quiz 3
1) 길이가 4096인 랜덤벡터를 하나 생성하세요.  `arange()`
2) 벡터를 위에서 배운 메소드들 중 하나를 이용해 (64x64) 정사각행렬로 차원을 조정하세요. `reshape() or view()` 
3) 1, 2의 과정을 반복해서 정사각행렬을 세 개 만든다음, 각각 텐서의 0번째 축에 크기가 1인 잉여차원을 추가하세요.  `unsqueeze()`
4) 0번째 축과 마지막 축을 교환하세요.   `transpose()`
5) 리스트에 세 개의 정사각행렬을 담아서 마지막 축을 기준으로 텐서를 합성하세요 -> 64x64x3 텐서가 나와야함  `cat()`
6) 위의 과정을 4번 반복해서 64x64x3 를 쌓아서 4 x 64 x 64 x 3 텐서를 만드세요  `stack()`

In [3]:
torch.manual_seed(1030)
# code : 
pass

NameError: name 'torch' is not defined

# Pop Quiz 4
1. Class.ipynb 에서 제시된 Pop Quiz에서 구현한 Policy Class의 torch의 구성요소로 재구현해보세요. 

#### Policy class requirements
- `__init__(self, state_dim, act_dim)`: 
  - 기능: state_dimension과 action_dimension을 입력으로 받아서 멤버변수로 저장합니다. 
  - [aciton_dim, state_dim]의 모양을 가지는 random matrix를 self.weight에 저장합니다. (Hint. `torch.randn` 메소드를 활용하세요)  
- `__call__(self, state)`:  
  - 기능: get_dist()를 내부적으로 호출하고 그 결과를 리턴합니다.  

- `get_dist(self, state)`:
  - 이 함수는 state를 입력받아서 distribution을 리턴합니다.   
  - self.weight와 state를 행렬곱을 통해 변환된 벡터를 얻습니다. 행렬곱은 `mat3 = mat1 @ mat2` 와 같은 방식으로 계산됩니다. 
  - 선형변환된 벡터를 nonlinear_vector = `relu(vector)`와 같은 방식으로 비선형함수를 통과시키세요. 
  - 계산된 벡터를 `softmax` 함수를 적용해서 확률분포 `probs`를 얻습니다.
  - `dist=Categorical(probs)` 를 통해 카테고리 분포 `dist`을 얻고 이를 리턴합니다.   
  
- `get_action(self, dist)`: 
  - 기능: distribution을 입력받아서 action_index를 샘플링하고 리턴합니다. 
  - `dist.sample()`를 통해 action_index를 리턴합니다. 

In [2]:
import torch
from torch.nn.functional import softmax, relu
from torch.distributions import Categorical

class Policy:
    def __init__(self, obs_dim, act_dim):
        pass
    
    def get_dist(self, state):
        pass
    
    def __call__(self, state):
        pass
    
    def get_action(self, dist):
        pass

In [3]:
obs_dim = 8
act_dim = 4
policy = Policy(obs_dim, act_dim)

state = torch.randn(8)
dist = policy(state)
action = policy.get_action(dist)
print(action)

None
